In [1]:
import cv2
import os

# Paths
video_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/videos"
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/frames"

# Ensure output directory exists
os.makedirs(frames_dir, exist_ok=True)

def extract_frames(video_path, output_dir, interval=1):
    """
    Extract frames from a video at a given interval (default: every 1 frame).
    """
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_count = 0
    video_name = os.path.basename(video_path).rsplit('.', 1)[0]  # Get the video name without extension

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every 'interval' frame
        if count % interval == 0:
            frame_path = os.path.join(output_dir, f"{video_name}_frame{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_count += 1

        count += 1

    cap.release()

# Process all videos
for video_file in os.listdir(video_dir):
    # Check if the filename ends with ".mp4" and starts with a valid number
    if video_file.endswith(".mp4"):
        try:
            # Split by '_' and attempt to convert the first part to float
            prefix = video_file.split('_')[0]
            float(prefix)  # This ensures it works for values like '0.5', '1', or '0'
            
            # Proceed with frame extraction
            video_path = os.path.join(video_dir, video_file)
            video_frames_dir = os.path.join(frames_dir, video_file.rsplit('.', 1)[0])  # Use full name
            os.makedirs(video_frames_dir, exist_ok=True)
            extract_frames(video_path, video_frames_dir)
            print(f"Frames extracted for {video_file}")
        except ValueError:
            print(f"Skipping file with invalid prefix: {video_file}")

print("Frame extraction completed!")

Frames extracted for 1_user5.mp4
Frames extracted for 1_user12.mp4
Frames extracted for 1_user6.mp4
Frames extracted for 1_user13.mp4
Frames extracted for 0_user2.mp4
Frames extracted for 0_user3.mp4
Frames extracted for 0_user19.mp4
Frames extracted for 0_user20.mp4
Frames extracted for 0_user10.mp4
Frames extracted for 1_user22.mp4
Frames extracted for 1_user23.mp4
Frames extracted for 0.5_user1.mp4
Frames extracted for 1_user21.mp4
Frames extracted for 1_user8.mp4
Frames extracted for 1_user9.mp4
Frame extraction completed!


In [2]:
import cv2
import os
import mediapipe as mp
import json
import math

# Paths
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/frames"
keypoints_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/keypoints"

# Ensure output directory exists
os.makedirs(keypoints_dir, exist_ok=True)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

def extract_keypoints_from_frame(frame_path):
    """
    Extract keypoints relevant to the javelin throwing motion.
    """
    image = cv2.imread(frame_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_image)

    if result.pose_landmarks:
        keypoints = {
            "right_hand": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].z,
            },
            "right_shoulder": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z,
            },
            "left_hand": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].z,
            },
            "left_shoulder": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z,
            },
            "pelvis_left": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].z,
            },
            "pelvis_right": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].z,
            },
            "torso": {
                "x": result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                "y": result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                "z": result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
            },
        }
        return keypoints
    return None

def check_pelvis_rotation(keypoints_sequence):
    """
    Check if the pelvis is rotated during the throwing motion (analyzing left and right hip position).
    """
    rotated_positions = []

    # Analyze pelvis rotation in the last 5 frames
    for keypoints in keypoints_sequence[-5:]:
        pelvis_left = keypoints["pelvis_left"]
        pelvis_right = keypoints["pelvis_right"]

        # Check if the left and right hips have shifted (x-coordinate comparison)
        if pelvis_left["x"] > pelvis_right["x"]:
            rotated_positions.append(True)
        else:
            rotated_positions.append(False)

    # Verify if the pelvis is consistently rotated in the last 5 frames
    return all(rotated_positions)

def check_javelin_fully_retracted(keypoints_sequence):
    """
    Check if the javelin is fully retracted during the last 5 steps.
    """
    fully_retracted = []

    # Analyze the last 5 frames
    for keypoints in keypoints_sequence[-5:]:
        right_hand = keypoints["right_hand"]
        right_shoulder = keypoints["right_shoulder"]

        # Check if the right hand is behind the right shoulder (x-coordinate comparison)
        if right_hand["x"] < right_shoulder["x"]:
            fully_retracted.append(True)
        else:
            fully_retracted.append(False)

    # Verify if the javelin is consistently retracted in the last 5 frames
    return all(fully_retracted)

def process_frames(video_frames_dir, output_path):
    """
    Process all frames for a video, detect pelvis rotation and javelin retraction, and save keypoints as JSON.
    """
    keypoints_data = []
    for frame_file in sorted(os.listdir(video_frames_dir)):
        frame_path = os.path.join(video_frames_dir, frame_file)
        keypoints = extract_keypoints_from_frame(frame_path)
        if keypoints:
            keypoints_data.append(keypoints)

    # Check for pelvis rotation and javelin retraction
    pelvis_rotation_detected = check_pelvis_rotation(keypoints_data)
    javelin_fully_retracted = check_javelin_fully_retracted(keypoints_data)

    # Save keypoints and detection status to JSON
    output_data = {
        "keypoints": keypoints_data,
        "pelvis_rotation_detected": pelvis_rotation_detected,
        "javelin_fully_retracted": javelin_fully_retracted,
    }
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=4)

# Process frames for each video
for video_name in os.listdir(frames_dir):
    video_frames_dir = os.path.join(frames_dir, video_name)
    if os.path.isdir(video_frames_dir):
        output_path = os.path.join(keypoints_dir, f"{video_name}_keypoints.json")
        process_frames(video_frames_dir, output_path)
        print(f"Keypoints, pelvis rotation, and javelin retraction detection completed for {video_name}")

print("Processing completed!")

I0000 00:00:1737121652.167359 6038652 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737121652.221196 6039441 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737121652.253168 6039446 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737121652.302850 6039444 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Keypoints, pelvis rotation, and javelin retraction detection completed for 0.5_user1
Keypoints, pelvis rotation, and javelin retraction detection completed for 0_user3
Keypoints, pelvis rotation, and javelin retraction detection completed for 0_user2
Keypoints, pelvis rotation, and javelin retraction detection completed for 0_user20
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_user13
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_user5
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_user22
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_user23
Keypoints, pelvis rotation, and javelin retraction detection completed for 0_user10
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_user12
Keypoints, pelvis rotation, and javelin retraction detection completed for 0_user19
Keypoints, pelvis rotation, and javelin retraction detection completed for 1_u

In [3]:
import os
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Paths
frames_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/frames"
clip_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/clip_features"
os.makedirs(clip_features_dir, exist_ok=True)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_clip_features(video_frames_dir, output_path, batch_size=8):
    """
    Extract CLIP features for all frames in a video using batch processing.
    """
    frame_features = []
    frame_paths = sorted(os.listdir(video_frames_dir))
    images = []

    for i, frame_file in enumerate(frame_paths):
        try:
            frame_path = os.path.join(video_frames_dir, frame_file)
            image = Image.open(frame_path).convert("RGB")
            images.append(image)

            # Process batch
            if len(images) == batch_size or i == len(frame_paths) - 1:
                inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
                with torch.no_grad():
                    image_features = model.get_image_features(**inputs).cpu().numpy()
                    frame_features.extend(image_features)
                images = []  # Clear batch to free memory
        except Exception as e:
            print(f"Error processing frame {frame_file}: {e}")
            continue

    # Save features
    torch.save(frame_features, output_path)

# Process all videos
for video_name in os.listdir(frames_dir):
    video_frames_dir = os.path.join(frames_dir, video_name)
    if os.path.isdir(video_frames_dir):
        output_path = os.path.join(clip_features_dir, f"{video_name}_clip.pt")
        extract_clip_features(video_frames_dir, output_path)
        print(f"CLIP features extracted for {video_name}")

print("CLIP feature extraction completed!")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CLIP features extracted for 0.5_user1
CLIP features extracted for 0_user3
CLIP features extracted for 0_user2
CLIP features extracted for 0_user20
CLIP features extracted for 1_user13
CLIP features extracted for 1_user5
CLIP features extracted for 1_user22
CLIP features extracted for 1_user23
CLIP features extracted for 0_user10
CLIP features extracted for 1_user12
CLIP features extracted for 0_user19
CLIP features extracted for 1_user8
CLIP features extracted for 1_user21
CLIP features extracted for 1_user6
CLIP features extracted for 1_user9
CLIP feature extraction completed!


In [4]:
import json
import os
import torch
import numpy as np
import math

# Paths
keypoints_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/keypoints"
clip_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/clip_features"
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/combined_features"

# Ensure output directory exists
os.makedirs(combined_features_dir, exist_ok=True)

# Function to calculate angle between three points (a, b, c) where b is the vertex
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_a = math.sqrt(ba[0] ** 2 + ba[1] ** 2)
    magnitude_b = math.sqrt(bc[0] ** 2 + bc[1] ** 2)
    
    cos_angle = dot_product / (magnitude_a * magnitude_b)
    angle = math.acos(cos_angle) * (180.0 / math.pi)
    
    return angle

# Detect pelvis rotation and javelin fully retracted position by analyzing keypoints
def detect_pelvis_rotation_and_javelin_retraction(keypoints_sequence):
    pelvis_rotation_detected = False
    javelin_retracted_detected = False

    for i in range(len(keypoints_sequence) - 5, len(keypoints_sequence)):
        try:
            # Focus on pelvis (hip) and arm positions for detecting rotation and retraction
            right_hip = keypoints_sequence[i]["right_shoulder"]  # Substitute with actual hip landmark if needed
            left_hip = keypoints_sequence[i]["left_shoulder"]  # Substitute with actual hip landmark if needed
            right_hand = keypoints_sequence[i]["right_hand"]
            left_hand = keypoints_sequence[i]["left_hand"]

            # Pelvis rotation: check if the pelvis has rotated by a certain angle (e.g., 30 degrees or more)
            pelvis_angle = calculate_angle(right_hip, left_hip, keypoints_sequence[i - 1]["right_hip"])
            if pelvis_angle > 30:  # Adjust this threshold based on the exact body motion
                pelvis_rotation_detected = True

            # Javelin retraction: check if the right hand is behind the right shoulder (indicating retraction)
            if right_hand["x"] < right_hip["x"]:  # Check if the hand is behind the hip in the x-axis
                javelin_retracted_detected = True

        except KeyError as e:
            print(f"KeyError: Missing key {e} in the keypoints data")

    return pelvis_rotation_detected, javelin_retracted_detected

# Combine pose-based features and CLIP embeddings for each video
def combine_features(video_name, pose_path, clip_path, output_path):
    with open(pose_path, 'r') as f:
        keypoints_data = json.load(f)["keypoints"]

    # Detect pelvis rotation and javelin retraction during the last steps
    pelvis_rotation, javelin_retracted = detect_pelvis_rotation_and_javelin_retraction(keypoints_data)

    # Load CLIP embeddings
    clip_data = torch.load(clip_path)

    # Combine features for each frame
    combined_data = []
    for i, clip_frame in enumerate(clip_data):
        pose_features = {
            "pelvis_rotation": 1 if pelvis_rotation else 0,  # Binary flag for pelvis rotation
            "javelin_retracted": 1 if javelin_retracted else 0,  # Binary flag for javelin retraction
        }
        combined_frame = np.concatenate([clip_frame, list(pose_features.values())])
        combined_data.append(combined_frame)

    # Save combined features
    torch.save(combined_data, output_path)

# Process all videos and combine features
for video_name in os.listdir(keypoints_dir):
    if video_name.endswith("_keypoints.json"):
        video_name_base = video_name.replace("_keypoints.json", "")
        pose_path = os.path.join(keypoints_dir, video_name)
        clip_path = os.path.join(clip_features_dir, f"{video_name_base}_clip.pt")
        output_path = os.path.join(combined_features_dir, f"{video_name_base}_combined.pt")

        if os.path.exists(clip_path):
            combine_features(video_name_base, pose_path, clip_path, output_path)
            print(f"Combined features saved for {video_name_base}")

print("Feature combination completed!")

KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
Combined features saved for 0_user3
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
Combined features saved for 1_user8
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
KeyError: Missing key 'right_hip' in the keypoints data
Combined features saved for 1_user23
KeyError: Missing key 'right_hip' in the keypoints 

/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_94727/3038619333.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clip_data = torch.load(clip_path)


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import os

# Paths
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/combined_features"

# Hyperparameters
sequence_length = 20  # Adjust as needed (still remains as sequence length)
batch_size = 16       # Adjust as needed

# Ensure input size is always 515
input_size = 515

class AthleticsDataset(Dataset):
    def __init__(self, combined_features_dir, sequence_length, input_size):
        """
        Handles loading and processing of combined features for athletics data.
        Ensures input size is always 515.
        """
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length
        self.input_size = input_size

        for file in os.listdir(combined_features_dir):
            if file.endswith("_combined.pt"):
                # Load combined features
                video_features = torch.tensor(torch.load(os.path.join(combined_features_dir, file)), dtype=torch.float32)

                # Truncate or pad sequences to the desired length
                if video_features.shape[0] >= self.sequence_length:
                    video_features = video_features[:self.sequence_length]
                else:
                    padding = torch.zeros((self.sequence_length - video_features.shape[0], video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=0)

                # Ensure features match the input size (515)
                if video_features.shape[1] != self.input_size:
                    padding = torch.zeros((video_features.shape[0], self.input_size - video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=1)
                self.data.append(video_features)

                # Extract label from filename
                label = float(file.split("_")[0])  # Extract label from file name
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Initialize dataset and DataLoader
def get_data_loaders(combined_features_dir, sequence_length, batch_size, input_size, train_split=0.8):
    dataset = AthleticsDataset(combined_features_dir, sequence_length, input_size)
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, input_size  # Return loaders and input size

# Create DataLoaders
train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length, batch_size, input_size)

print(f"DataLoaders created. Input size: {input_size}")

DataLoaders created. Input size: 515


/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_94727/3694048949.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  video_features = torch.tensor(torch.load(os.

In [6]:
import torch.nn as nn

class TemporalModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        """
        LSTM-based model for sequence prediction.
        """
        super(TemporalModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)  # Use the final hidden state
        output = self.fc(hidden[-1])  # Fully connected output
        return output

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import itertools
import os

# Paths
combined_features_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/combined_features"

# Dataset and DataLoader
class AthleticsDataset(torch.utils.data.Dataset):
    def __init__(self, combined_features_dir, sequence_length):
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length

        for file in os.listdir(combined_features_dir):
            if file.endswith("_combined.pt"):
                video_features = torch.tensor(torch.load(os.path.join(combined_features_dir, file)), dtype=torch.float32)

                if video_features.shape[0] >= self.sequence_length:
                    video_features = video_features[:self.sequence_length]
                else:
                    padding = torch.zeros((self.sequence_length - video_features.shape[0], video_features.shape[1]))
                    video_features = torch.cat((video_features, padding), dim=0)

                self.data.append(video_features)
                label = float(file.split("_")[0])  # Extract label from file name
                self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def get_data_loaders(combined_features_dir, sequence_length, batch_size, train_split=0.8):
    dataset = AthleticsDataset(combined_features_dir, sequence_length)
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, dataset[0][0].shape[1]

# Define LSTM model
class TemporalModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(TemporalModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return output

# Training function with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=500, patience=100):
    best_val_loss = float("inf")
    patience_counter = 0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), torch.tensor(labels, dtype=torch.float32).to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), torch.tensor(labels, dtype=torch.float32).to(device)
                outputs = model(features)
                loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # # Early stopping logic with 100 epochs patience
        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     patience_counter = 0
        # else:
        #     patience_counter += 1

        # if patience_counter >= patience:
        #     print(f"Early stopping triggered after {patience} epochs without improvement!")
        #     break

    return best_val_loss


# Hyperparameter grid search
# def hyperparameter_search():
    # Hyperparameter grid
    hidden_sizes = [32, 64, 128]
    num_layers = [1, 2, 3]
    learning_rates = [0.001, 0.005, 0.0001, 0.0005]
    dropouts = [0.0, 0.2, 0.3]

    # Initialize variables to track the best model
    best_val_loss = float("inf")
    best_params = None
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Prepare data
    train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length=20, batch_size=8)

    # Iterate over all combinations of hyperparameters
    for hidden_size, num_layer, learning_rate, dropout in itertools.product(hidden_sizes, num_layers, learning_rates, dropouts):
        print(f"Testing configuration: Hidden Size={hidden_size}, Num Layers={num_layer}, LR={learning_rate}, Dropout={dropout}")
        
        # Initialize model, criterion, and optimizer
        model = TemporalModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layer, output_size=1, dropout=dropout).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Train the model
        val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, device)

        # Update best model if this configuration is better
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = {
                "hidden_size": hidden_size,
                "num_layers": num_layer,
                "learning_rate": learning_rate,
                "dropout": dropout,
            }

    print(f"Best Configuration: {best_params}, Validation Loss: {best_val_loss:.4f}")
    return best_params

# Run hyperparameter search
# best_params = hyperparameter_search()

In [12]:
# Train the final model with the best configuration
final_hidden_size = 128
final_num_layers = 1
final_learning_rate = 0.0012
final_dropout = 0.5

# Prepare DataLoaders (use full dataset for training)
train_loader, val_loader, input_size = get_data_loaders(combined_features_dir, sequence_length=20, batch_size=32)
print(f"Input size for the model: {input_size}")

# Initialize final model
device = "cuda" if torch.cuda.is_available() else "cpu"
final_model = TemporalModel(input_size=input_size, hidden_size=final_hidden_size, num_layers=final_num_layers, output_size=1, dropout=final_dropout).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=final_learning_rate)

# Train final model
final_val_loss = train_model(final_model, train_loader, val_loader, criterion, optimizer, device, num_epochs=1000, patience=100)
print(f"Final model trained. Validation Loss: {final_val_loss:.4f}")

/var/folders/fb/p7zywq8d7351kf4l4fwh2dmw0000gn/T/ipykernel_94727/1542848075.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  video_features = torch.tensor(torch.load(os.

Input size for the model: 514
Epoch [1/1000], Train Loss: 0.6913, Val Loss: 0.1854
Epoch [2/1000], Train Loss: 0.2938, Val Loss: 0.3344
Epoch [3/1000], Train Loss: 0.2538, Val Loss: 0.4546
Epoch [4/1000], Train Loss: 0.2950, Val Loss: 0.4143
Epoch [5/1000], Train Loss: 0.2543, Val Loss: 0.3243
Epoch [6/1000], Train Loss: 0.1980, Val Loss: 0.2489
Epoch [7/1000], Train Loss: 0.1651, Val Loss: 0.2032
Epoch [8/1000], Train Loss: 0.1564, Val Loss: 0.1806
Epoch [9/1000], Train Loss: 0.1586, Val Loss: 0.1714
Epoch [10/1000], Train Loss: 0.1601, Val Loss: 0.1696
Epoch [11/1000], Train Loss: 0.1554, Val Loss: 0.1732
Epoch [12/1000], Train Loss: 0.1442, Val Loss: 0.1824
Epoch [13/1000], Train Loss: 0.1287, Val Loss: 0.1983
Epoch [14/1000], Train Loss: 0.1123, Val Loss: 0.2232
Epoch [15/1000], Train Loss: 0.0981, Val Loss: 0.2590
Epoch [16/1000], Train Loss: 0.0881, Val Loss: 0.3042
Epoch [17/1000], Train Loss: 0.0822, Val Loss: 0.3517
Epoch [18/1000], Train Loss: 0.0784, Val Loss: 0.3901
Epoch [

In [13]:
import matplotlib.pyplot as plt

def evaluate_and_collect_predictions(model, data_loader, device):
    """
    Evaluate the model on the provided DataLoader and collect true vs. predicted values.
    """
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for features, labels in data_loader:
            features = features.to(device)
            labels = labels.to(device)

            # Get model predictions
            outputs = model(features).squeeze()
            predicted_labels.extend(outputs.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return true_labels, predicted_labels

# Use validation set for evaluation
true_labels, predicted_labels = evaluate_and_collect_predictions(final_model, val_loader, device)


In [14]:
import cv2
import os
import torch
from PIL import Image
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import mediapipe as mp

# Paths for javelin throwing
video_dir = "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/videos"
sequence_length = 20  # Sequence length for LSTM

# Initialize CLIP and MediaPipe
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# Ensure input size is always 515 for javelin throwing
input_size = 514

# Load your model (final_model should be defined elsewhere)
# final_model = ...  # Make sure to load your trained model here

# Process and Predict for each video
def process_and_predict(video_path, model, sequence_length):
    print(f"Processing video: {os.path.basename(video_path)}")
    
    # Step 1: Extract frames from video
    frames = []
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % 5 == 0:  # Extract frame every 5th frame
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(rgb_frame)
        count += 1
    cap.release()

    # Step 2: Extract pose keypoints using MediaPipe
    keypoints = []
    for frame in frames:
        result = pose.process(frame)
        if result.pose_landmarks:
            keypoints.append([
                {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                for lm in result.pose_landmarks.landmark
            ])
    
    # Step 3: Detect large stride before the throw (run-up phase)
    run_up_stride_scores = []
    for i in range(1, len(keypoints)):
        if i < len(keypoints):  # Check bounds
            lead_leg_prev = keypoints[i - 1][25]  # Right knee (Lead leg)
            lead_leg_curr = keypoints[i][25]  # Right knee (Lead leg)

            # Calculate stride length (distance between two consecutive positions of lead leg)
            stride_length = np.sqrt((lead_leg_curr['x'] - lead_leg_prev['x'])**2 + (lead_leg_curr['y'] - lead_leg_prev['y'])**2)

            # Define a large stride threshold based on experimentation or domain knowledge
            large_stride_threshold = 0.05  # Adjust this threshold based on your requirements
            run_up_stride_scores.append(stride_length > large_stride_threshold)

    # Step 4: Analyze shoulder, elbow, and wrist alignment during the throw (post-throw mechanics)
    throw_alignment_scores = []
    for i in range(len(keypoints)):
        if i < len(keypoints):  # Check bounds
            shoulder = keypoints[i][11]  # Left shoulder (Torso)
            elbow = keypoints[i][13]    # Left elbow
            wrist = keypoints[i][15]    # Left wrist

            # Vectors for shoulder -> elbow and elbow -> wrist
            shoulder_to_elbow = np.array([elbow['x'] - shoulder['x'], elbow['y'] - shoulder['y']])
            elbow_to_wrist = np.array([wrist['x'] - elbow['x'], wrist['y'] - elbow['y']])

            # Calculate the angle between shoulder-elbow and elbow-wrist (angle of throw)
            angle = np.arctan2(elbow_to_wrist[1], elbow_to_wrist[0]) - np.arctan2(shoulder_to_elbow[1], shoulder_to_elbow[0])
            throw_alignment_scores.append(np.degrees(angle))

    # Step 5: Detect backward motion during the last 5 frames (Javelin brought backward)
    backward_motion_scores = []
    hand_positions = [keypoints[i][15] for i in range(len(keypoints))]  # Track the wrist (dominant hand)

    for i in range(len(hand_positions) - 5, len(hand_positions)):
        if i > 0:  # Ensure we are not accessing out of bounds
            prev_position = hand_positions[i - 1]
            curr_position = hand_positions[i]

            # Calculate displacement in X, Y (backward motion detection)
            displacement = np.sqrt((curr_position['x'] - prev_position['x'])**2 + (curr_position['y'] - prev_position['y'])**2)
            # Determine if the hand moved backward (negative movement along x-axis)
            backward_motion_scores.append(curr_position['x'] < prev_position['x'])

    # Step 6: Extract CLIP embeddings for frames (semantic features)
    clip_embeddings = []
    for frame in frames:
        image = Image.fromarray(frame)
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            embedding = clip_model.get_image_features(**inputs).cpu().numpy().flatten()
            clip_embeddings.append(embedding)

    # Step 7: Combine all features (CLIP, stride, throw alignment, backward motion)
    combined_features = []
    for i, clip_embedding in enumerate(clip_embeddings):
        feature_vector = np.concatenate([clip_embedding, 
                                        [run_up_stride_scores[i] if i < len(run_up_stride_scores) else 0],
                                        [throw_alignment_scores[i] if i < len(throw_alignment_scores) else 0],
                                        [backward_motion_scores[i] if i < len(backward_motion_scores) else 0]])
        
        if len(feature_vector) > input_size:
            feature_vector = feature_vector[:input_size]  # Truncate if too long
        elif len(feature_vector) < input_size:
            padding = np.zeros(input_size - len(feature_vector))  # Pad with zeros if too short
            feature_vector = np.concatenate([feature_vector, padding])
        combined_features.append(feature_vector)
    
    # Convert to tensor
    combined_features = torch.tensor(combined_features, dtype=torch.float32)

    # Step 8: Truncate or pad sequences to the fixed sequence length
    if combined_features.shape[0] >= sequence_length:
        combined_features = combined_features[:sequence_length]
    else:
        padding = torch.zeros((sequence_length - combined_features.shape[0], combined_features.shape[1]))
        combined_features = torch.cat((combined_features, padding), dim=0)

    # Step 9: Predict using the model
    model.eval()
    with torch.no_grad():
        combined_features = combined_features.unsqueeze(0).to(device)  # Add batch dimension
        prediction = model(combined_features).squeeze().cpu().item()

    print(f"Prediction: {prediction:.4f}")
    return prediction


# Process all videos in the directory and predict
results = {}
for video_file in os.listdir(video_dir):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(video_dir, video_file)
        prediction = process_and_predict(video_path, final_model, sequence_length)
        results[video_file] = prediction

# Print results
print("\nPrediction Results:")
for video, prediction in results.items():
    print(f"{video}: {prediction:.4f}")

I0000 00:00:1737121772.707069 6038652 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
W0000 00:00:1737121772.766003 6045024 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737121772.805769 6045024 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing video: 1_user5.mp4
Prediction: 0.8274
Processing video: 1_user12.mp4
Prediction: 0.8670
Processing video: 1_user6.mp4
Prediction: 0.9539
Processing video: 1_user13.mp4
Prediction: 0.6803
Processing video: 0_user2.mp4
Prediction: 0.6615
Processing video: 0_user3.mp4
Prediction: 0.8220
Processing video: 0_user19.mp4
Prediction: 0.5842
Processing video: 0_user20.mp4
Prediction: 0.6684
Processing video: 0_user10.mp4
Prediction: 0.6120
Processing video: 1_user22.mp4
Prediction: 0.9202
Processing video: 1_user23.mp4
Prediction: 0.7648
Processing video: 0.5_user1.mp4
Prediction: 0.8721
Processing video: 1_user21.mp4
Prediction: 0.8991
Processing video: 1_user8.mp4
Prediction: 0.9425
Processing video: 1_user9.mp4
Prediction: 0.9611

Prediction Results:
1_user5.mp4: 0.8274
1_user12.mp4: 0.8670
1_user6.mp4: 0.9539
1_user13.mp4: 0.6803
0_user2.mp4: 0.6615
0_user3.mp4: 0.8220
0_user19.mp4: 0.5842
0_user20.mp4: 0.6684
0_user10.mp4: 0.6120
1_user22.mp4: 0.9202
1_user23.mp4: 0.7648
0.5_use

In [17]:
torch.save(final_model.state_dict(), "/Users/danyukezz/Desktop/2 year 1 semester/team project/danya_preprocessing_sports/spear_throwing/stages/stage2/stage2_javelin.pth")

___

## feedback generation

In [ ]:
def process_video_with_metadata(video_path, model, sequence_length):
    """
    Process a video and return structured metadata with predictions and pose metrics.
    """
    print(f"Processing video: {os.path.basename(video_path)}")
    
    # Step 1: Extract frames
    frames = extract_frames(video_path)
    
    # Step 2: Extract keypoints
    keypoints = extract_keypoints(frames)

    # Step 3: Calculate pose-based features
    velocities, accelerations = calculate_velocity_and_acceleration(keypoints)
    stride_lengths = calculate_stride_length(keypoints)

    # Pose metrics
    pose_metrics = {
        "average_velocity": np.mean(velocities) if velocities else 0,
        "max_stride_length": np.max(stride_lengths) if stride_lengths else 0,
        "average_acceleration": np.mean(accelerations) if accelerations else 0
    }

    # Step 4: Extract CLIP features
    clip_embeddings = extract_clip_features(frames)

    # CLIP insights
    clip_summary = "High contextual alignment with accelerating motion."  # Placeholder for now

    # Step 5: Combine features
    combined_features = combine_features(clip_embeddings, velocities, accelerations, stride_lengths)

    # Step 6: Truncate or pad sequences
    if combined_features.shape[0] >= sequence_length:
        combined_features = combined_features[:sequence_length]
    else:
        padding = torch.zeros((sequence_length - combined_features.shape[0], combined_features.shape[1]))
        combined_features = torch.cat((combined_features, padding), dim=0)

    # Step 7: Predict using the model
    model.eval()
    with torch.no_grad():
        combined_features = combined_features.unsqueeze(0).to(device)  # Add batch dimension
        prediction = model(combined_features).squeeze().cpu().item()

    print(f"Prediction: {prediction:.4f}")

    # Return structured metadata
    return {
        "video_name": os.path.basename(video_path),
        "prediction": prediction,
        "pose_metrics": pose_metrics,
        "clip_features": {"embedding_summary": clip_summary}
    }

In [ ]:
# from transformers import pipeline

# generator = pipeline("text2text-generation", model="google/flan-t5-base")
# result = generator("Explain the performance of an athlete based on metrics.", max_length=50)
# print(result[0]['generated_text'])


# def validate_metadata(metadata):
#     required_keys = {
#         'video_name': str,
#         'prediction': float,
#         'pose_metrics': dict,
#         'clip_features': dict,
#     }
#     pose_metrics_keys = ['average_velocity', 'max_stride_length', 'average_acceleration']
#     clip_features_keys = ['embedding_summary']

#     for key, expected_type in required_keys.items():
#         if key not in metadata or not isinstance(metadata[key], expected_type):
#             raise ValueError(f"Invalid or missing key: {key}, expected type: {expected_type}")
    
#     for key in pose_metrics_keys:
#         if key not in metadata['pose_metrics']:
#             raise ValueError(f"Missing pose metric: {key}")
    
#     for key in clip_features_keys:
#         if key not in metadata['clip_features']:
#             raise ValueError(f"Missing clip feature: {key}")

# def generate_justification(metadata, max_length=150, num_return_sequences=1):
#     validate_metadata(metadata)
    
#     # Dynamic prompt construction
#     metrics_prompt = []
#     for metric, value in metadata['pose_metrics'].items():
#         metrics_prompt.append(f"- {metric.replace('_', ' ').title()}: {value:.2f}")
#     metrics_text = "\n".join(metrics_prompt)

#     prompt = f"""
#     Analyze the performance for {metadata['video_name']}.
#     The predicted score is {metadata['prediction']:.2f}.
#     Key metrics:
#     {metrics_text}
#     - CLIP embedding summary: {metadata['clip_features']['embedding_summary']}
    
#     Based on these metrics, explain why the score is appropriate and provide constructive feedback for improvement.
#     """
#     print("Generated prompt:", prompt)  # Debugging log
#     result = generator(prompt, max_length=max_length, num_return_sequences=num_return_sequences)
#     return result[0]['generated_text']


In [ ]:
# # Directory containing videos
# video_dir = "/Users/cezar/Desktop/Team Project/AI/distance_jump/stage1/videos"

# # Dictionary to store results
# results = {}

# for video_file in os.listdir(video_dir):
#     if video_file.endswith(".mp4"):
#         video_path = os.path.join(video_dir, video_file)
        
#         # Step 1: Process video and generate metadata
#         metadata = process_video_with_metadata(video_path, final_model, sequence_length=20)
        
#         # Step 2: Generate justification using Hugging Face model
#         justification = generate_justification(metadata)
#         metadata["justification"] = justification
        
#         # Store results
#         results[video_file] = metadata

# # Print results
# for video, data in results.items():
#     print(f"Video: {video}")
#     print(f"Prediction: {data['prediction']:.4f}")
#     print(f"Justification: {data['justification']}")
#     print("-" * 50)

# # Optional: Save results to JSON
# # import json
# # with open("video_predictions_with_justifications.json", "w") as f:
# #     json.dump(results, f, indent=4)